In [11]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

#### 1. ConversationBufferMemory（对话缓存记忆）
这种记忆允许存储消息，然后从变量中提取消息。

In [12]:
import warnings
warnings.filterwarnings('ignore')
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory = memory,
                                 verbose=True#查看Langchain实际上在做什么，设为FALSE的话只给出回答，看到不到下面绿色的内容
)


In [13]:
conversation.predict(input="Hi, my name is andy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is andy
AI:

> Finished chain.


'Hello, Andy! How can I assist you today? 😊'

In [14]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today? 😊
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1 plus 1 is 2. \n\n**Answer:** 1 + 1 equals 2.'

In [15]:
print(memory.buffer)

Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today? 😊
Human: What is 1+1?
AI: The sum of 1 plus 1 is 2. 

**Answer:** 1 + 1 equals 2.


In [16]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is andy\nAI: Hello, Andy! How can I assist you today? 😊\nHuman: What is 1+1?\nAI: The sum of 1 plus 1 is 2. \n\n**Answer:** 1 + 1 equals 2.'}

#### 2. ConversationBufferWindowMemory（对话缓存窗口记忆）
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。

In [18]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)     # k=1表明只保留一个对话记忆    
conversation = ConversationChain(llm=llm, memory = memory,
    verbose=True    #这里改为FALSE不显示提示，你可以尝试修改为TRUE后的结果
)

In [19]:
conversation.predict(input="Hi, my name is andy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is andy
AI:

> Finished chain.


'Hello, Andy! How can I assist you today?'

In [20]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is andy
AI: Hello, Andy! How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The sum of 1 and 1 is 2. \n\n**Step-by-Step Explanation:**\n\n1. Start with the number 1.\n2. Add another 1 to it.\n3. The total is 2.\n\nThus, 1 + 1 equals 2.'

In [21]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The sum of 1 and 1 is 2. 

**Step-by-Step Explanation:**

1. Start with the number 1.
2. Add another 1 to it.
3. The total is 2.

Thus, 1 + 1 equals 2.
Human: What is my name?
AI:

> Finished chain.


"I don't have a name, but I'm here to help you with whatever you need!"

#### 3. ConversationTokenBufferMemory（对话token缓存记忆）
使用对话token缓存记忆，内存将限制保存的token数量。如果token数量超出指定数目，它会切掉这个对话的早期部分 以保留与最近的交流相对应的token数量，但不超过token限制。

In [22]:
pip install -q tiktoken 

Note: you may need to restart the kernel to use updated packages.


In [4]:
import warnings
warnings.filterwarnings('ignore')
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import httpx
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0.0,api_key=os.environ['OPENAI_API_KEY'],base_url="https://api-inference.modelscope.cn/v1/", http_client=httpx.Client(verify=False),model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

In [9]:
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10 # 限制最大10个token
                                      )
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [10]:
memory.load_memory_variables({})

{'history': 'AI: Charming!'}

#### 4. ConversationSummaryBufferMemory（对话摘要缓存记忆）
这种Memory的想法是，不是将内存限制为基于最近对话的固定数量的token或固定数量的对话次数窗口，而是使用LLM编写到目前为止历史对话的摘要，并将其保存

In [11]:
from langchain.memory import ConversationSummaryBufferMemory
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)   #使用对话摘要缓存记忆
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [12]:
memory.load_memory_variables({})

{'history': 'System: The human asks what the AI thinks of artificial intelligence. The AI thinks AI is a force for good because it will help humans reach their full potential. The human greets the AI, they exchange casual greetings, and the human asks about the AI\'s schedule. The AI responds with "Cool."\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [14]:
from langchain.chains import ConversationChain
conversation = ConversationChain(llm=llm, memory = memory, verbose=True)

In [15]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks what the AI thinks of artificial intelligence. The AI thinks AI is a force for good because it will help humans reach their full potential. The human greets the AI, they exchange casual greetings, and the human asks about the AI's schedule. The AI responds with "Cool."
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your

'That sounds like a fantastic idea! A fun and interactive demo would definitely make a great impression. How about setting up a mini "AI assistent" game or challenge at the restaurant? You could create a fun, interactive experience where the AI demonstrates its problem-solving skills, like playing a quick game of 20 Questions, or helping with a fun trivia quiz. Alternatively, you could showcase an AI-powered interactive art installation or a creative project that brings AI to life in a playful way. The key is to make it engaging and show how AI can add value to everyday experiences. What do you think?'

In [16]:
memory.load_memory_variables({})   #摘要记录更新了

{'history': 'System: The human asks what the AI thinks of artificial intelligence. The AI thinks AI is a force for good because it will help humans reach their full potential. The human greets the AI, they exchange casual greetings, and the human asks about the AI\'s schedule. The AI responds with "Cool." The AI provides a detailed schedule: a meeting at 8am with the product team, 9am-12pm working on the LangChain project, a noon lunch at an Italian restaurant with a customer, and a demo idea for a fun and interactive experience like a mini "AI assistant" game or an AI-powered interactive art installation. The human asks for a good demo, and the AI suggests ideas like a game of 20 Questions or a trivia quiz to showcase AI\'s problem-solving skills.'}